# crawler

> crawler with call backs

In [2]:
#| default_exp crawler

In [3]:
#| hide
from nbdev.showdoc import *

In [16]:
#| export 
from pw.core import  *
from operator import attrgetter
import inspect
from fastcore.all import *
import asyncio

In [11]:
#| export 
#| hide
class Callback(): order = 0

async def run_cbs(cbs, method_nm, crawler=None, *args, **kwargs):
    for cb in sorted(cbs, key=attrgetter('order')):
        method = getattr(cb, method_nm, None)
        if method :
            if inspect.iscoroutinefunction(method):
                await method(crawler, *args, **kwargs)
            else:
                method(crawler, *args, **kwargs)

class with_cbs:
    def __init__(self, nm): self.nm = nm
    def __call__(self, f):
        def _f(o, *args, **kwargs):
            try:
                o.callback(f'before_{self.nm}')
                print(self.nm)
                f(o, *args, **kwargs)
                o.callback(f'after_{self.nm}')
            except globals()[f'Cancel{self.nm.title()}Exception']: pass
            finally: o.callback(f'cleanup_{self.nm}')
        return _f

In [22]:
#| export
class Crawl():
    def __init__(self, np: int = 1, to_visit: Optional[List[str]] = None, cbs=None): # type: ignore
        self.np = np
        self.visited = set()
        self.unvisited = set(to_visit)
        self.cbs = L(cbs)
        
    async def one_visit(self, idx):
        page = self.pages[idx]
        
        await self.callback('before_visit',idx)
        
        await page.goto(self.visit_window[idx])
        await page.wait()

        await self.callback('after_visit', idx)
        
    async def run(self, headless:bool = False):
        async with setup_browser(n=self.np, headless=headless) as obj:
            if obj.is_valid:
                    self.pages = obj.pages

                    while self.unvisited:
                        #print(self.unvisited, self.visited)
                        self.visit_window = list(self.unvisited - self.visited)[:self.np] #not visited 
                        
                        tasks = [self.one_visit(i) for i in range(len(self.visit_window)) ]
                    
                        await asyncio.gather(*tasks)  
                        visited_urls = set(self.visit_window)
                        self.unvisited.difference_update(visited_urls) # remove the urls from the to_visit
                        self.visited.update(visited_urls)

    def __getattr__(self, name):
        if name.startswith('before_') or name.startswith('after_'): return partial(self.callback, name)
        raise AttributeError(name)

    async def callback(self, method_nm, *args, **kwargs ): 
        await run_cbs(self.cbs, method_nm, self, *args, **kwargs)          

In [29]:
class CB(Callback):
    def __init__(self):
        print(f"__init__ " )

    def before_visit(self, crawler, idx):
        print(f"before_visit  {idx=}" )

    def after_visit(self, crawler, idx):
        print(f"before_visit  {idx=}" )

C = Crawl(1, ['https://solveit.fast.ai/', 'https://fastcore.fast.ai/'], [CB()])
await C.run(False)

__init__ 
before_visit  idx=0
before_visit  idx=0
before_visit  idx=0
before_visit  idx=0


In [30]:
C = Crawl(2, ['https://solveit.fast.ai/', 'https://fastcore.fast.ai/'], [CB()])
await C.run(False)

__init__ 
before_visit  idx=0
before_visit  idx=1
before_visit  idx=0
before_visit  idx=1


In [31]:
#| hide
import nbdev; nbdev.nbdev_export()